In [ ]:
""" 下载数据集 """
import dataHandler

dataHandler.download_data_from_baostock(start_date="2023-08-30", end_date="2024-05-20", adjustflag = "1")
dataHandler.transform_csv_into_bin("2023-08-30~2024-05-20")

In [ ]:
""" 初始化参数 """
import qlib
from qlib import data
from qlib.utils import init_instance_by_config

# 数据集初始化
qlib.init(provider_uri = "D:/PythonProject/rich/stock_data/qlib_cn_data_from_baostock/2023-08-30~2024-05-20")
stock_list = data.D.instruments(market = "all")

# 数据处理器参数配置：整体数据开始结束时间，训练集开始结束时间，股票池
data_handler_config = {
    "start_time": "2023-08-30",
    "end_time": "2024-05-20",
    "fit_start_time": "2023-08-30",
    "fit_end_time": "2024-02-19",
    "instruments": stock_list
}

# 任务参数配置
task = {
    # 机器学习模型参数配置
    "model": {
        "class": "LGBModel", # 模型类
        "module_path": "qlib.contrib.model.gbdt", # 模型类所在的模块
        "kwargs": { # 模型类超参数配置，使用这些参数初始化上面使用的模型类对象
            "loss": "mse",
            "colsample_bytree": 0.8879,
            "learning_rate": 0.0421,
            "subsample": 0.8789,
            "lambda_l1": 205.6999,
            "lambda_l2": 580.9768,
            "max_depth": 8,
            "num_leaves": 210,
            "num_threads": 20,
        },
    },
    # 因子数据参数配置
    "dataset": {
        "class": "DatasetH", # 数据集类，是Dataset with Data(H)andler的缩写，即带数据处理器的数据集
        "module_path": "qlib.data.dataset", # 数据集类所在的模块
        "kwargs": { # 数据集类的参数配置
            "handler": { # 数据集使用的数据处理器配置
                "class": "Alpha158", # 数据处理器类，继承自DataHandlerLP，特征
                "module_path": "qlib.contrib.data.handler", # 数据处理器类所在模块
                "kwargs": data_handler_config, # 数据处理器参数配置
            },
            "segments": { # 数据集时段划分
                "train": ("2023-08-30", "2024-02-19"), # 训练集时段
                "valid": ("2024-02-20", "2024-3-20"), # 验证集时段
                "test": ("2024-3-21", "2024-05-20"),  # 测试集时段
            },
        },
    },
}

# 实例化模型对象
model = init_instance_by_config(task["model"])
dataset = init_instance_by_config(task["dataset"])

In [ ]:
""" 执行训练模型 """
from qlib.workflow import R
from qlib.utils import flatten_dict

with R.start(experiment_name="train"):
    R.log_params(**flatten_dict(task))
    model.fit(dataset)
    R.save_objects(**{"trained_model.pkl": model})
    print('info:', R.get_recorder().info)